In [1]:
import time

import numpy as np
import pandas as pd
import seaborn as sns
import datetime as dt
import xgboost as xgb

from itertools import combinations

import matplotlib.pyplot as plt
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 24, 8
%matplotlib inline

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler, LabelEncoder, OneHotEncoder

sns.set_context(context='notebook')

In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from numba import jit
import time
import gc

In [3]:
MAX_ROUNDS = 250 #220
OPTIMIZE_ROUNDS = False
LEARNING_RATE = 0.07
EARLY_STOPPING_ROUNDS = 50  

In [4]:
# Compute gini
@jit
def eval_gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

def gini_xgb(preds, dtrain):
    labels = dtrain.get_label()
    gini_score = -eval_gini(labels, preds)
    return [('gini', gini_score)]

In [5]:
def recon_ps_reg_03(reg):
    integer = int(np.round((40*reg)**2)) # gives 2060 for our example
    for f in range(28):
        if (integer - f) % 27 == 0:
            F = f
    M = (integer - F)//27
    return F, M

In [6]:
def add_noise(series, noise_level):
    return series * (1 + noise_level * np.random.randn(len(series)))

In [7]:
def target_encode(trn_series=None,
                  tst_series=None,
                  target=None,
                  min_samples_leaf=1,
                  smoothing=1,
                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_tst_series, noise_level)

In [8]:
def target_encode_with_validation(trn_series=None,    # Revised to encode validation series
                                  val_series=None,
                                  tst_series=None,
                                  target=None,
                                  min_samples_leaf=1,
                                  smoothing=1,
                                  noise_level=0):
    """
    Smoothing is computed like in the following paper by Daniele Micci-Barreca
    https://kaggle2.blob.core.windows.net/forum-message-attachments/225952/7441/high%20cardinality%20categoricals.pdf
    trn_series : training categorical feature as a pd.Series
    tst_series : test categorical feature as a pd.Series
    target : target data as a pd.Series
    min_samples_leaf (int) : minimum samples to take category average into account
    smoothing (int) : smoothing effect to balance categorical average vs prior
    """
    assert len(trn_series) == len(target)
    assert trn_series.name == tst_series.name
    temp = pd.concat([trn_series, target], axis=1)
    # Compute target mean
    averages = temp.groupby(by=trn_series.name)[target.name].agg(["mean", "count"])
    # Compute smoothing
    smoothing = 1 / (1 + np.exp(-(averages["count"] - min_samples_leaf) / smoothing))
    # Apply average function to all target data
    prior = target.mean()
    # The bigger the count the less full_avg is taken into account
    averages[target.name] = prior * (1 - smoothing) + averages["mean"] * smoothing
    averages.drop(["mean", "count"], axis=1, inplace=True)
    # Apply averages to trn and tst series
    ft_trn_series = pd.merge(
        trn_series.to_frame(trn_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=trn_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_trn_series.index = trn_series.index
    ft_val_series = pd.merge(
        val_series.to_frame(val_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=val_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_val_series.index = val_series.index
    ft_tst_series = pd.merge(
        tst_series.to_frame(tst_series.name),
        averages.reset_index().rename(columns={'index': target.name, target.name: 'average'}),
        on=tst_series.name,
        how='left')['average'].rename(trn_series.name + '_mean').fillna(prior)
    # pd.merge does not keep the index so restore it
    ft_tst_series.index = tst_series.index
    return add_noise(ft_trn_series, noise_level), add_noise(ft_val_series, noise_level), add_noise(ft_tst_series, noise_level)

In [9]:
# Read data
train_df = pd.read_csv('../data/train.csv', na_values="-1") # .iloc[0:200,:]
test_df = pd.read_csv('../data/test.csv', na_values="-1")

In [10]:
train_features = [
    "ps_car_13",  #            : 1571.65 / shadow  609.23
	"ps_reg_03",  #            : 1408.42 / shadow  511.15
	"ps_ind_05_cat",  #        : 1387.87 / shadow   84.72
	"ps_ind_03",  #            : 1219.47 / shadow  230.55
	"ps_ind_15",  #            :  922.18 / shadow  242.00
	"ps_reg_02",  #            :  920.65 / shadow  267.50
	"ps_car_14",  #            :  798.48 / shadow  549.58
	"ps_car_12",  #            :  731.93 / shadow  293.62
	"ps_car_01_cat",  #        :  698.07 / shadow  178.72
	"ps_car_07_cat",  #        :  694.53 / shadow   36.35
	"ps_ind_17_bin",  #        :  620.77 / shadow   23.15
	"ps_car_03_cat",  #        :  611.73 / shadow   50.67
	"ps_reg_01",  #            :  598.60 / shadow  178.57
	"ps_car_15",  #            :  593.35 / shadow  226.43
	"ps_ind_01",  #            :  547.32 / shadow  154.58
	"ps_ind_16_bin",  #        :  475.37 / shadow   34.17
	"ps_ind_07_bin",  #        :  435.28 / shadow   28.92
	"ps_car_06_cat",  #        :  398.02 / shadow  212.43
	"ps_car_04_cat",  #        :  376.87 / shadow   76.98
	"ps_ind_06_bin",  #        :  370.97 / shadow   36.13
	"ps_car_09_cat",  #        :  214.12 / shadow   81.38
	"ps_car_02_cat",  #        :  203.03 / shadow   26.67
	"ps_ind_02_cat",  #        :  189.47 / shadow   65.68
	"ps_car_11",  #            :  173.28 / shadow   76.45
	"ps_car_05_cat",  #        :  172.75 / shadow   62.92
	"ps_calc_09",  #           :  169.13 / shadow  129.72
	"ps_calc_05",  #           :  148.83 / shadow  120.68
	"ps_ind_08_bin",  #        :  140.73 / shadow   27.63
	"ps_car_08_cat",  #        :  120.87 / shadow   28.82
	"ps_ind_09_bin",  #        :  113.92 / shadow   27.05
	"ps_ind_04_cat",  #        :  107.27 / shadow   37.43
	"ps_ind_18_bin",  #        :   77.42 / shadow   25.97
	"ps_ind_12_bin",  #        :   39.67 / shadow   15.52
	"ps_ind_14",  #            :   37.37 / shadow   16.65
    #'ps_reg_F',
    #'ps_reg_M'
]


In [11]:
combs = [
    ('ps_reg_01', 'ps_car_02_cat'),  
    ('ps_reg_01', 'ps_car_04_cat'),
    #('ps_reg_F', 'ps_reg_M'),
]

In [12]:
# # Add 'ps_reg_03' reconstruction
# train_df['ps_reg_03'] = train_df['ps_reg_03'].fillna(-1)
# test_df['ps_reg_03'] = test_df['ps_reg_03'].fillna(-1)

# train_df['ps_reg_F'] = train_df['ps_reg_03'].apply(lambda x: recon_ps_reg_03(x)[0] if x!=-1 else -1)
# train_df['ps_reg_M'] = train_df['ps_reg_03'].apply(lambda x: recon_ps_reg_03(x)[1] if x!=-1 else -1)

# test_df['ps_reg_F'] = test_df['ps_reg_03'].apply(lambda x: recon_ps_reg_03(x)[0] if x!=-1 else -1)
# test_df['ps_reg_M'] = test_df['ps_reg_03'].apply(lambda x: recon_ps_reg_03(x)[1] if x!=-1 else -1)

In [13]:
# Process data
id_test = test_df['id'].values
id_train = train_df['id'].values
y = train_df['target']

start = time.time()
for n_c, (f1, f2) in enumerate(combs):
    name1 = f1 + "_plus_" + f2
    print('\nCurrent feature %60s %4d in %5.1f'
          % (name1, n_c + 1, (time.time() - start) / 60), end='')
    print('\r' * 75, end='')
    train_df[name1] = train_df[f1].apply(lambda x: str(x)) + "_" + train_df[f2].apply(lambda x: str(x))
    test_df[name1] = test_df[f1].apply(lambda x: str(x)) + "_" + test_df[f2].apply(lambda x: str(x))
    # Label Encode
    lbl = LabelEncoder()
    lbl.fit(list(train_df[name1].values) + list(test_df[name1].values))
    train_df[name1] = lbl.transform(list(train_df[name1].values))
    test_df[name1] = lbl.transform(list(test_df[name1].values))

    train_features.append(name1)


Current feature                                 ps_reg_01_plus_ps_car_02_cat    1 in   0.0
Current feature                                 ps_reg_01_plus_ps_car_04_cat    2 in   0.0

In [14]:
X = train_df[train_features]
test_df = test_df[train_features]

f_cats = [f for f in X.columns if "_cat" in f]

In [15]:
for f in f_cats:
    X[f + "_avg"], test_df[f + "_avg"] = target_encode(trn_series=X[f],
                                                         tst_series=test_df[f],
                                                         target=y,
                                                         min_samples_leaf=200,
                                                         smoothing=10,
                                                         noise_level=0)

/home/laith/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


In [16]:
X = X.fillna(-1)
test_df = test_df.fillna(-1)

In [17]:
y_valid_pred = 0*y
y_test_pred = 0

In [18]:
K = 5
kf = KFold(n_splits = K, random_state = 1, shuffle = True)
np.random.seed(42)

In [19]:
def test_rand(train, labels):
    # Create the pipeline
    # Create a parameter grid to search for best parameters for everything in the pipeline
    param_grid = {
                    #'min_child_weight':np.random.randint(10,size=(1, 10)), # due to high class imbalance
                    'objective': ['binary:logistic'],
                    'max_depth': [4],
                    'max_delta_step': [1.8],
                    'colsample_bytree': [0.8],
                    'subsample': [0.8],
                    'learning_rate': np.linspace(0.15,0.8,num=70),#np.random.uniform(0.02,.07,1000),#[0.028084362074445819], #0.025
                    'gamma': [10],#np.random.uniform(7,12,20), #np.random.uniform(0.066,.07,1000),
                    'n_estimators' : [375],
                    'reg_alpha':np.linspace(6,20,num=25),#[0.01],
                    'reg_lambda':np.linspace(1,2.6,num=25),
                    'tree_method': ['gpu_exac'],
                    'colsample_bylevel':[0.432],
                    'min_child_weight':np.linspace(18,35,num=25),
                    'silent': [1],
                    'missing':[-1],
                    'n_jobs': [12],
                    #'updater': 'grow_gpu',
                    'scale_pos_weight' :  np.linspace(1,2.6,num=25), # due to high class imbalance
                  }

    # Normalized Gini Scorer
    #gini_scorer = metrics.make_scorer(normalized_gini, greater_is_better = True)
    params = {'tree_method':'gpu_hist', 'predictor':'gpu_predictor'}
    # Initialize Grid Search Model
    model = RandomizedSearchCV(estimator  = xgb.XGBClassifier(**params),
                         param_distributions = param_grid,
                         #param_grid = param_grid,
                         scoring    = 'roc_auc',
                         n_iter     = 400,     
                         verbose    = 2,
                         n_jobs     = 12,
                         iid        = True,
                         refit      = True,
                         cv         = StratifiedKFold( n_splits=3, shuffle=True))
    # Fit Grid Search Model
    model.fit(train, labels)
    print("Best score: %0.3f" % model.best_score_)
    print("Best parameters set:")
    best_parameters = model.best_estimator_.get_params()
    for param_name in sorted(param_grid.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

    # Get best model
    best_model = model.best_estimator_

    # Fit model with best parameters optimized for normalized_gini
    #best_model.fit(train,labels)

    return best_model, model

In [20]:
t = time.time()
bstmdl, grd = test_rand(X,y)
print()
print('this took {}-seconds'.format(time.time()-t))

Fitting 3 folds for each of 400 candidates, totalling 1200 fits
[CV] tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.26666666667, reg_lambda=2.6, reg_alpha=17.6666666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=24.375, max_depth=4, max_delta_step=1.8, learning_rate=0.705797101449, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432 
[CV] tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.26666666667, reg_lambda=2.6, reg_alpha=17.6666666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=24.375, max_depth=4, max_delta_step=1.8, learning_rate=0.705797101449, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432 
[CV] tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.26666666667, reg_lambda=2.6, reg_alpha=17.6666666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=24.375, max_depth=4, max_delta_step=1.8, learning_

[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.0, reg_lambda=1.93333333333, reg_alpha=20.0, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=30.0416666667, max_depth=4, max_delta_step=1.8, learning_rate=0.705797101449, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432, total= 1.6min
[CV] tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.93333333333, reg_lambda=1.26666666667, reg_alpha=6.0, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=30.75, max_depth=4, max_delta_step=1.8, learning_rate=0.197101449275, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432 
[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.2, reg_lambda=1.26666666667, reg_alpha=6.58333333333, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=21.5416666667, max_depth=4, max_delta_step=1.8, learning_rate=0.536231884058, gamma=10, colsample_byt

[Parallel(n_jobs=12)]: Done  17 tasks      | elapsed:  3.2min


[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.33333333333, reg_lambda=1.53333333333, reg_alpha=7.16666666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=25.7916666667, max_depth=4, max_delta_step=1.8, learning_rate=0.790579710145, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432, total= 1.6min
[CV] tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=2.53333333333, reg_lambda=1.0, reg_alpha=10.6666666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=23.6666666667, max_depth=4, max_delta_step=1.8, learning_rate=0.78115942029, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432 
[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.33333333333, reg_lambda=1.53333333333, reg_alpha=7.16666666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=25.7916666667, max_depth=4, max_delta_step=1.8, learning_rate=0.7

[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.2, reg_lambda=1.46666666667, reg_alpha=17.6666666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=24.375, max_depth=4, max_delta_step=1.8, learning_rate=0.724637681159, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432, total= 1.5min
[CV] tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.0, reg_lambda=2.6, reg_alpha=15.9166666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=34.2916666667, max_depth=4, max_delta_step=1.8, learning_rate=0.394927536232, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432 
[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=2.53333333333, reg_lambda=1.0, reg_alpha=10.6666666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=23.6666666667, max_depth=4, max_delta_step=1.8, learning_rate=0.78115942029, gamma=10, colsample_bytr

...

[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.0, reg_lambda=1.13333333333, reg_alpha=17.0833333333, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=28.625, max_depth=4, max_delta_step=1.8, learning_rate=0.178260869565, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432, total= 1.7min
[CV] tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=2.33333333333, reg_lambda=1.4, reg_alpha=6.0, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=28.625, max_depth=4, max_delta_step=1.8, learning_rate=0.319565217391, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432 
[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.26666666667, reg_lambda=1.4, reg_alpha=11.25, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=30.0416666667, max_depth=4, max_delta_step=1.8, learning_rate=0.639855072464, gamma=10, colsample_bytree=0.8, colsam

[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=2.33333333333, reg_lambda=1.4, reg_alpha=6.0, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=28.625, max_depth=4, max_delta_step=1.8, learning_rate=0.319565217391, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432, total= 1.6min
[CV] tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.93333333333, reg_lambda=1.66666666667, reg_alpha=10.0833333333, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=34.2916666667, max_depth=4, max_delta_step=1.8, learning_rate=0.611594202899, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432 
[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.8, reg_lambda=2.2, reg_alpha=12.4166666667, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=29.3333333333, max_depth=4, max_delta_step=1.8, learning_rate=0.234782608696, gamma=10, colsample_byt

[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.66666666667, reg_lambda=1.73333333333, reg_alpha=13.5833333333, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=35.0, max_depth=4, max_delta_step=1.8, learning_rate=0.385507246377, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432, total= 1.1min
[CV]  tree_method=gpu_exac, subsample=0.8, silent=1, scale_pos_weight=1.66666666667, reg_lambda=1.73333333333, reg_alpha=13.5833333333, objective=binary:logistic, n_jobs=12, n_estimators=375, missing=-1, min_child_weight=35.0, max_depth=4, max_delta_step=1.8, learning_rate=0.385507246377, gamma=10, colsample_bytree=0.8, colsample_bylevel=0.432, total=  48.9s


[Parallel(n_jobs=12)]: Done 1200 out of 1200 | elapsed: 163.0min finished


Best score: 0.640
Best parameters set:
	colsample_bylevel: 0.432
	colsample_bytree: 0.8
	gamma: 10
	learning_rate: 0.17826086956521739
	max_delta_step: 1.8
	max_depth: 4
	min_child_weight: 28.625
	missing: -1
	n_estimators: 375
	n_jobs: 12
	objective: 'binary:logistic'
	reg_alpha: 17.083333333333336
	reg_lambda: 1.1333333333333333
	scale_pos_weight: 1.0
	silent: 1
	subsample: 0.8
	tree_method: 'gpu_hist'

this took 9795.313354969025-seconds
